In [4]:
import pathlib

import osmnx

from clusters import Cluster, ClusterCentreStrategy
from population import PopulationGeneratorConfig, generate_data_points
from traffic import IncrementalBatchRouteAssigner
from travel import TravelGenerator
import pandas as pd


RESOLUTION = 8


config = PopulationGeneratorConfig.from_json_file(pathlib.Path("/home/ajwo/IdeaProjects/slupsk-user-equilibrium/configs/slupsk_2004.json"))
cluster_centre_strategy = ClusterCentreStrategy.HEXAGON_CENTER
pts_by_epi = {epi.label: generate_data_points(epi) for epi in config.epicentres}
points = [point for pts in pts_by_epi.values() for point in pts]
places = [
    'Słupsk, Polska',
    'gmina Słupsk, Polska',
    'gmina Kobylnica, Polska',
]
custom_filter = '["highway"~"motorway|primary|secondary|tertiary"]'
graph = osmnx.graph.graph_from_place(places, network_type="drive", custom_filter=custom_filter)
graph = osmnx.speed.add_edge_travel_times(osmnx.add_edge_speeds(graph))
clusters = Cluster.consolidate_clusters(Cluster.clusterize_points(points, RESOLUTION, cluster_centre_strategy), graph, RESOLUTION)
travels = TravelGenerator(config).generate_travels(clusters)
assigner = IncrementalBatchRouteAssigner(h3_resolution=8, iterations_count=5, batch_size=100)

In [5]:
routes = assigner.assign_routes(travels, clusters, graph)

In [12]:
pd.DataFrame([
    {
        "start": start, 
        "end": end, 
        "travel_time": state.travel_time.minutes, 
        "free_flow_travel_time": state.path_data.free_flow_travel_time.minutes, 
        "capacity": state.path_data.max_capacity, 
        "volume": len(state.current_volume)
    } for (start, end, state) in assigner.graph.edges.data("state")
])

,start,end,travel_time,free_flow_travel_time,capacity,volume
0,881f0d4547fffff,881f0d450dfffff,1.622127,1.515424,2200,1821
1,881f0d4547fffff,881f0d4509fffff,1.794330,1.791804,2200,685
2,881f0d4547fffff,881f0d4541fffff,0.942393,0.941985,2200,510
3,881f0d4547fffff,881f0d456bfffff,2.554377,1.806216,2200,2836
4,881f0d4547fffff,881f0d4545fffff,2.797235,2.525329,2200,2025
...,...,...,...,...,...,...
101,881f0d4097fffff,881f0d456dfffff,2.011231,1.613957,2200,2490
102,881f0d40b3fffff,881f0d456dfffff,1.018184,1.017644,4400,1073
103,881f0d4091fffff,881f0d4097fffff,1.727804,1.727804,2200,18
104,881f0d4091fffff,881f0d409bfffff,0.845541,0.845541,2200,0


In [ ]:

df = pd.DataFrame([{
    "start": route.travel.start.h3_hex_id, 
    "end": route.travel.end.h3_hex_id, 
    "travel_time_in_minutes": route.estimated_travel_time.minutes,
    "free_flow_travel_time_in_minutes": route
} for route in routes])

In [ ]:
stats = pd.DataFrame({"travel_count": df.groupby(["start", "end"]).size(), "travel_time": df.groupby(["start", "end"])["travel_time_in_minutes"].mean()})

In [ ]:
stats